In [1]:
# 该代码用于处理第三批病人的SE数据 利用课题组提供的H5黑盒模型

In [2]:
# 引入工具包
from mne.io import concatenate_raws,read_raw_fif
import matplotlib.pyplot as plt
import mne
import tensorflow as tf
import numpy as np
from numpy.lib.utils import safe_eval
from keras.models import load_model
import keras.backend as K
import glob
import scipy.signal
import scipy.io as scio
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
def pearson_r(y_true, y_pred):
    x = y_true
    y = y_pred
    mx = K.mean(x, axis=0)
    my = K.mean(y, axis=0)
    xm, ym = x - mx, y - my
    r_num = K.sum(xm * ym)
    x_square_sum = K.sum(xm * xm)
    y_square_sum = K.sum(ym * ym)
    r_den = K.sqrt(x_square_sum * y_square_sum)
    r = r_num / r_den
    return K.mean(r)
# % 模型准备 %
model_path = r'../others/Gao_Novel_CNN_RNN16_2561630689427.h5'
model = load_model(model_path, custom_objects={'pearson_r': pearson_r})

In [4]:
def SE_process(file_path,save_path,select_time):
    raw = read_raw_fif(file_path)
    #改成了15通道
    data,times=raw[:15,:]
    # select_time = 4 #抽取2s的片段，可修改为偶数秒  
    
    samps = int(select_time * 256)
    
    # 重采样为256Hz
    data_256 = []
    for i in range(data.shape[0]):
            data_temp = scipy.signal.resample(data[i],samps)
            data_256.append(data_temp)
    # 重新转换为np.array
    data_256 = np.array(data_256)                                    
    print("data_256.shape",data_256.shape)
    
    # 分成 epoches，256*2的形状
    data_256_samples = []
    # 对每个通道
    for i in range(data_256.shape[0]):
        # 进行片段切片
        channel = data_256[i];
        # samps = 2*256 两秒，每秒256个采样点
        data_channels = []
        for j in range(0,channel.shape[0],2*256):
            # print("第" + str(j//samps) + "个片段")
            data_temp = channel[j:j+2*256]
            data_channels.append(data_temp)
        # 每个通道整合起来
        data_256_samples.append(data_channels)

    # 转回np.array
    data_256_samples = np.array(data_256_samples)
    print("data_256_samples.shape",data_256_samples.shape)

    # 进行标准化处理,sta表示Standardization
    data_sta = []
    # 对每个通道
    for i in range(data_256_samples.shape[0]):
        # np.mean和np.std计算均值和标准差
        tmp_mean = np.mean(data_256_samples[i])
        tmp_std = np.std(data_256_samples[i])
        data_temp_std = (data_256_samples[i] - tmp_mean) / tmp_std
        data_sta.append(data_temp_std)
    # 转回np.array
    data_sta = np.array(data_sta)
    print("data_sta.shape",data_sta.shape) 

    # 整理成神经网络训练所需的形状，这里把数据展平成一维，所有通道合并，之后再还原
    data_x = data_sta.reshape(-1,512,1)
    print("data_x.shape:",data_x.shape)
    # 模型进行预测
    y_pred = model.predict(data_x, verbose=0)
    # 先把通道分开
    # data_256_samples.shape[1] = 30 即降采样后的片段数
    
    pred_channel = []

    for i in range(0,y_pred.shape[0],data_256_samples.shape[1]):
        channel_temp = y_pred[i:i+data_256_samples.shape[1]]
        channel_temp = channel_temp.flatten()
        pred_channel.append(channel_temp)

    pred_channel = np.array(pred_channel)
    print("pred_channel.shape",pred_channel.shape)
    
    #改成了15通道
    info = mne.create_info(ch_names=raw.ch_names[:15],ch_types=["eeg"]*15,sfreq=256)
    """
    利用mne.io.RawArray类创建Raw对象
    """
    custom_raw = mne.io.RawArray(data=pred_channel, info=info)
    custom_raw.save(save_path,overwrite=True)
    
    
    return True

In [5]:
# file_path = "./se(已筛选)/SE(1)_eeg.fif"
# save_path = "./SE(processed)/"
# SE_process(file_path,save_path)

In [6]:
train_directory = glob.glob("../SE_train/"+"*")
test_directory = glob.glob("../SE_test/"+"*")
print(len(train_directory),len(test_directory))

257 28


In [7]:
for file_path in train_directory:
    file_name = file_path.split("\\")[-1]
    save_path = "../SE_train_processed/" + "".join(file_name)
    print(save_path)
    #训练集为8s
    SE_process(file_path,save_path,8)
    
for file_path in test_directory:
    file_name = file_path.split("\\")[-1]
    save_path = "../SE_test_processed/" + "".join(file_name)
    print(save_path)
    #测试集为16s
    SE_process(file_path,save_path,16)

../SE_train_processed/Z何富桂20220221_bias_0.fif
Opening raw data file ../SE_train\Z何富桂20220221_bias_0.fif...
Isotrak not found
    Range : 8919 ... 17111 =      8.710 ...    16.710 secs
Ready.


<ipython-input-4-d8650b052227>:2: RuntimeWarning: This filename (../SE_train\Z何富桂20220221_bias_0.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw = read_raw_fif(file_path)


data_256.shape (15, 2048)
data_256_samples.shape (15, 4, 512)
data_sta.shape (15, 4, 512)
data_x.shape: (60, 512, 1)
pred_channel.shape (15, 2048)
Creating RawArray with float64 data, n_channels=15, n_times=2048
    Range : 0 ... 2047 =      0.000 ...     7.996 secs
Ready.
Writing E:\jupyter file\EEG_process\SE_train_processed\Z何富桂20220221_bias_0.fif
Closing E:\jupyter file\EEG_process\SE_train_processed\Z何富桂20220221_bias_0.fif
[done]
../SE_train_processed/Z何富桂20220225_bias_0.fif
Opening raw data file ../SE_train\Z何富桂20220225_bias_0.fif...
Isotrak not found
    Range : 233098 ... 241290 =    227.635 ...   235.635 secs
Ready.
data_256.shape (15, 2048)
data_256_samples.shape (15, 4, 512)
data_sta.shape (15, 4, 512)
data_x.shape: (60, 512, 1)
pred_channel.shape (15, 2048)
Creating RawArray with float64 data, n_channels=15, n_times=2048
    Range : 0 ... 2047 =      0.000 ...     7.996 secs
Ready.
Writing E:\jupyter file\EEG_process\SE_train_processed\Z何富桂20220225_bias_0.fif
Closing E:\jup

<ipython-input-4-d8650b052227>:74: RuntimeWarning: This filename (E:\jupyter file\EEG_process\SE_train_processed\Z何富桂20220221_bias_0.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  custom_raw.save(save_path,overwrite=True)
<ipython-input-4-d8650b052227>:2: RuntimeWarning: This filename (../SE_train\Z何富桂20220225_bias_0.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw = read_raw_fif(file_path)
<ipython-input-4-d8650b052227>:74: RuntimeWarning: This filename (E:\jupyter file\EEG_process\SE_train_processed\Z何富桂20220225_bias_0.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.f

[done]
../SE_train_processed/Z何富桂20220228_bias_0.fif
Opening raw data file ../SE_train\Z何富桂20220228_bias_0.fif...
Isotrak not found
    Range : 10580 ... 18772 =     10.332 ...    18.332 secs
Ready.
data_256.shape (15, 2048)
data_256_samples.shape (15, 4, 512)
data_sta.shape (15, 4, 512)
data_x.shape: (60, 512, 1)
pred_channel.shape (15, 2048)
Creating RawArray with float64 data, n_channels=15, n_times=2048
    Range : 0 ... 2047 =      0.000 ...     7.996 secs
Ready.
Writing E:\jupyter file\EEG_process\SE_train_processed\Z何富桂20220228_bias_0.fif
Closing E:\jupyter file\EEG_process\SE_train_processed\Z何富桂20220228_bias_0.fif
[done]
../SE_train_processed/Z何富桂20220302_bias_0.fif
Opening raw data file ../SE_train\Z何富桂20220302_bias_0.fif...
Isotrak not found


...

<ipython-input-4-d8650b052227>:74: RuntimeWarning: This filename (E:\jupyter file\EEG_process\SE_test_processed\Z陈琳20220513（未发作）.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  custom_raw.save(save_path,overwrite=True)
<ipython-input-4-d8650b052227>:2: RuntimeWarning: This filename (../SE_test\Z陈琳20220516.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw = read_raw_fif(file_path)


pred_channel.shape (15, 4096)
Creating RawArray with float64 data, n_channels=15, n_times=4096
    Range : 0 ... 4095 =      0.000 ...    15.996 secs
Ready.
Writing E:\jupyter file\EEG_process\SE_test_processed\Z陈琳20220516.fif
Closing E:\jupyter file\EEG_process\SE_test_processed\Z陈琳20220516.fif
[done]
../SE_test_processed/Z陈琳20220518（未发作）.fif
Opening raw data file ../SE_test\Z陈琳20220518（未发作）.fif...
Isotrak not found
    Range : 146 ... 32914 =      0.071 ...    16.071 secs
Ready.
data_256.shape (15, 4096)
data_256_samples.shape (15, 8, 512)
data_sta.shape (15, 8, 512)
data_x.shape: (120, 512, 1)


<ipython-input-4-d8650b052227>:74: RuntimeWarning: This filename (E:\jupyter file\EEG_process\SE_test_processed\Z陈琳20220516.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  custom_raw.save(save_path,overwrite=True)
<ipython-input-4-d8650b052227>:2: RuntimeWarning: This filename (../SE_test\Z陈琳20220518（未发作）.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw = read_raw_fif(file_path)


pred_channel.shape (15, 4096)
Creating RawArray with float64 data, n_channels=15, n_times=4096
    Range : 0 ... 4095 =      0.000 ...    15.996 secs
Ready.
Writing E:\jupyter file\EEG_process\SE_test_processed\Z陈琳20220518（未发作）.fif
Closing E:\jupyter file\EEG_process\SE_test_processed\Z陈琳20220518（未发作）.fif
[done]


<ipython-input-4-d8650b052227>:74: RuntimeWarning: This filename (E:\jupyter file\EEG_process\SE_test_processed\Z陈琳20220518（未发作）.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  custom_raw.save(save_path,overwrite=True)
